In [1]:
import sys
sys.path.append("../script/")

import Functions
from Engine import Engine
from GTGP import GTGP


from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import importlib
from time import time
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import numpy as np
import pandas as pd
from time import time

In [2]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils import shuffle
def plot_roc_curve(true_y, y_prob):
    """
    plots the roc curve based of the probabilities
    """
    
    fpr, tpr, thresholds = roc_curve(true_y, y_prob)
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    print(roc_auc_score(true_y,prob[:,1]))

# data

In [30]:
df = pd.read_csv("../data/fars.tsv",delimiter='\t')
X = df.iloc[:,:-1].to_numpy().astype("float")
y = df.iloc[:,-1].to_numpy().astype("int")

seeds = [10086, 200,500,30506,30405,30420,10056,7059,40965,5398,869543,83491,823190,
         48392,2810,48392,3498210,483902,859032,12890,538920,86954,54309,6504,9840,
         219805,548,2981,432890,5438908,219094,5843902,60854,979,12890,2108,4093]
train_size = 0.7
dataset = 'fars'

X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=train_size,stratify=y)

In [6]:
def to_one_hot(y,yt):
    le = LabelEncoder()
    ohe = OneHotEncoder()
    y_train = le.fit_transform(y)
    y_one_hot = ohe.fit_transform(y_train.reshape(-1,1))
    
    y_test = le.transform(yt)
    yt_one_hot = ohe.transform(y_test.reshape(-1,1))
    
    return y_train,y_test,y_one_hot,yt_one_hot
y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)

# model

In [35]:
def fit_trees():
    learning_rate=0.1
    max_depth=24
    bins=8
    lam=1

    gtgp = GTGP(learning_rate=learning_rate,max_depth=max_depth,bins=bins,lam=lam)

    total_size=10
    elite_size = 10
    epoch= 100
    gp_epoch= 3
    verbose = 1
    tolerance=0.001

    gtgp.fit(X_train,y_train,total_size=total_size,elite_size = elite_size,epoch=epoch,gp_epoch=gp_epoch,tolerance=tolerance,verbose=verbose)

    retrain_epoch=5
    alpha=1
    beta=1
    gammer=1

    verbose=1
    gtgp.lam = 1
    gtgp.retrain_estimators(X_test,y_test,retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)

    return gtgp

for i in range(1):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    gtgp = fit_trees()

    print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
    print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))

    num_trees,depth,num_nodes = gtgp.print_model()

    train_acc = accuracy_score(y_train,np.argmax(gtgp.train_p,axis=1))
    test_acc = accuracy_score(y_test,np.argmax(gtgp.test_p,axis=1))
    train_f1 = f1_score(y_train,np.argmax(gtgp.train_p,axis=1),average='macro')
    test_f1 = f1_score(y_test,np.argmax(gtgp.test_p,axis=1),average='macro')


    with open('./benchmark/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

30 118 1
59 229 2
89 353 3
118 468 4
146 574 5
171 663 6
199 771 7
227 887 8
251 991 9
277 1099 10
304 1222 11
330 1324 12
353 1411 13
378 1510 14
404 1634 15
428 1746 16
451 1849 17
471 1931 18
494 2030 19
517 2115 20
538 2204 21
561 2307 22
583 2389 23
608 2488 24
625 2549 25
646 2644 26
667 2741 27
691 2837 28
710 2912 29
731 3011 30
753 3107 31
775 3205 32
789 3255 33
805 3321 34
822 3390 35
843 3471 36
866 3588 37
884 3670 38
907 3787 39
925 3869 40
945 3969 41
963 4043 42
979 4113 43
1000 4198 44
1015 4261 45
1030 4332 46
1050 4424 47
1065 4497 48
1082 4580 49
1098 4644 50
1114 4730 51
1131 4809 52
1149 4905 53
1165 4983 54
1177 5033 55
1197 5135 56
1211 5197 57
1228 5272 58
1247 5357 59
1261 5423 60
1277 5501 61
1296 5604 62
1308 5656 63
1328 5744 64
1350 5850 65
1360 5892 66
1378 5988 67
1395 6059 68
1411 6129 69
1429 6227 70
1445 6301 71
1459 6357 72
1477 6433 73
1491 6505 74
1509 6601 75
1528 6728 76
1541 6787 77
1555 6849 78
1571 6941 79
1586 7018 80
1597 7065 81
1614 7138 8

KeyboardInterrupt: 

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    clf = DecisionTreeClassifier()
    clf.fit(X_train,y_train)

    num_trees = 1
    depth = clf.tree_.max_depth
    num_nodes = clf.tree_.node_count

    train_acc = accuracy_score(y_train,clf.predict(X_train))
    test_acc = accuracy_score(y_test,clf.predict(X_test))
    train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
    test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')

    with open('./benchmark_DC/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# XGBoost

In [24]:
import xgboost

In [31]:
with open('./benchmark_xgb/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        # xgb = xgboost.XGBClassifier(n_estimators=100,max_depth=6)
        # xgb = xgboost.XGBClassifier(min_child_weight=0,max_depth=4)
        xgb = xgboost.XGBClassifier()
        xgb.fit(X_train,y_train)

        import json

        def item_generator(json_input, lookup_key):
            if isinstance(json_input, dict):
                for k, v in json_input.items():
                    if k == lookup_key:
                        yield v
                    else:
                        yield from item_generator(v, lookup_key)
            elif isinstance(json_input, list):
                for item in json_input:
                    yield from item_generator(item, lookup_key)

        def tree_depth(json_text):
            json_input = json.loads(json_text)
            depths = list(item_generator(json_input, 'depth'))
            return max(depths) + 1 if len(depths) != 0 else 1

        train_acc = accuracy_score(y_train,xgb.predict(X_train))
        test_acc = accuracy_score(y_test,xgb.predict(X_test))

        train_f1 = f1_score(y_train,xgb.predict(X_train),average='macro')
        test_f1 = f1_score(y_test,xgb.predict(X_test),average='macro')

        y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
        # train_f1 = roc_auc_score(y_train_one_hot.toarray(),xgb.predict_proba(X_train))
        # test_f1 = roc_auc_score(y_test_one_hot.toarray(),xgb.predict_proba(X_test))

        booster = xgb.get_booster()

        tree_df = booster.trees_to_dataframe()
        depths = [tree_depth(x) for x in booster.get_dump(dump_format = "json")]
        num_trees = len(depths)
        depth = np.average(depths)
        num_nodes = len(tree_df)

        print(classification_report(y_train,xgb.predict(X_train),digits=5))
        print(classification_report(y_test,xgb.predict(X_test),digits=5))

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000         6
           1    0.99990   0.99885   0.99937     29481
           2    0.68341   0.70303   0.69308     10550
           3    0.88827   0.76077   0.81959       209
           4    0.86919   0.98922   0.92533     14005
           5    0.55433   0.66533   0.60478      9723
           6    0.68634   0.21838   0.33133      6072
           7    0.83796   0.86054   0.84910       631

    accuracy                        0.83791     70677
   macro avg    0.81492   0.77451   0.77782     70677
weighted avg    0.83674   0.83791   0.82543     70677

              precision    recall  f1-score   support

           0    0.00000   0.00000   0.00000         3
           1    0.99992   0.99858   0.99925     12635
           2    0.60198   0.62008   0.61089      4522
           3    0.56061   0.41111   0.47436        90
           4    0.85441   0.98167   0.91363      6002
           5    0.47673 

c:\Users\zhouz\anaconda3\envs\BStackGP\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zhouz\anaconda3\envs\BStackGP\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zhouz\anaconda3\envs\BStackGP\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000         6
           1    0.99993   0.99881   0.99937     29481
           2    0.67845   0.70218   0.69011     10550
           3    0.89062   0.81818   0.85287       209
           4    0.86787   0.98915   0.92455     14005
           5    0.55379   0.67551   0.60863      9723
           6    0.73175   0.19812   0.31182      6072
           7    0.84365   0.86371   0.85356       631

    accuracy                        0.83761     70677
   macro avg    0.82076   0.78071   0.78011     70677
weighted avg    0.83964   0.83761   0.82382     70677

              precision    recall  f1-score   support

           0    0.00000   0.00000   0.00000         3
           1    0.99992   0.99858   0.99925     12635
           2    0.61026   0.63954   0.62455      4522
           3    0.60317   0.42222   0.49673        90
           4    0.85406   0.98384   0.91437      6002
           5    0.47892 

c:\Users\zhouz\anaconda3\envs\BStackGP\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zhouz\anaconda3\envs\BStackGP\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\zhouz\anaconda3\envs\BStackGP\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


KeyboardInterrupt: 

# GDBT

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    clf = GradientBoostingClassifier()
    # clf = GradientBoostingClassifier()
    clf.fit(X_train,y_train)

    train_acc = accuracy_score(y_train,clf.predict(X_train))
    test_acc = accuracy_score(y_test,clf.predict(X_test))
    train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
    test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')

    num_trees = len([ est for ests in clf.estimators_ for est in ests])
    depth = np.average([ max(1,est.tree_.max_depth) for ests in clf.estimators_ for est in ests])
    num_nodes = sum([ est.tree_.node_count for ests in clf.estimators_ for est in ests])

    with open('./benchmark_GBDT/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    rfc = RandomForestClassifier()
    rfc.fit(X_train,y_train)

    train_acc = accuracy_score(y_train,rfc.predict(X_train))
    test_acc = accuracy_score(y_test,rfc.predict(X_test))
    train_f1 = f1_score(y_train,rfc.predict(X_train),average='macro')
    test_f1 = f1_score(y_test,rfc.predict(X_test),average='macro')

    num_trees = len(rfc.estimators_)
    depth = np.average([est.tree_.max_depth for est in rfc.estimators_])
    num_nodes = sum([est.tree_.node_count for est in rfc.estimators_])

    with open('./benchmark_RF/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)